<a href="https://colab.research.google.com/github/Waelalessa21/transfer-learning-pytorch/blob/main/transfer_learning_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 Project

<details>
<summary>Project Details (click to expand)</summary>

## Project Goal
The goal of this project is to apply **transfer learning** using a pretrained deep learning model and study how different transfer learning strategies perform under **small data** and **large data**.

## Model Used
- **ResNet-18**, pretrained on ImageNet

## Datasets Used

### 1. Small Dataset (~100 images)
- **Dataset:** Cats vs Dogs
- **Source:** Kaggle
- **URL:** https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset

### 2. Large Dataset (10,000+ images)
- **Dataset:** Intel Image Classification
- **Source:** Kaggle
- **URL:** https://www.kaggle.com/datasets/puneet6060/intel-image-classification

## Transfer Learning Strategies
For each dataset, two transfer learning techniques are applied:

1. **Freeze Backbone**
   - All pretrained convolutional layers are frozen
   - Only the newly added classifier (model head) is trained

2. **Fine-tune Entire Model**
   - All layers of the pretrained model are retrained
   - Allows full adaptation to the target dataset

## Evaluation Criteria
- Training time
- Validation accuracy

</details>


# Kaggle API Setup

Setting the environment variables to allow Colab to download datasets from Kaggle

In [1]:
import os

os.environ["KAGGLE_USERNAME"] = "waelalessa"
os.environ["KAGGLE_KEY"] = "KGAT_f42d187651e38b9f18a4b372bb2e40d5"


# Small Dataset (100 images)

- Dataset: Cats vs Dogs
- Source: [Kaggle](https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset)
- Task: Binary classification (Cat vs Dog)
- Purpose: Observe transfer learning performance with very limited data

## Download and Prepare Small Dataset



In [2]:
!mkdir -p ./data
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset -p ./data
!unzip -q ./data/microsoft-catsvsdogs-dataset.zip -d ./data/cats_vs_dogs

Dataset URL: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset
License(s): other
microsoft-catsvsdogs-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace ./data/cats_vs_dogs/MSR-LA - 3467.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


## import required libraries

In [3]:
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

## Define transformation

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


## Load full dataset (100)

In [5]:
full_dataset = datasets.ImageFolder(root='/content/data/cats_vs_dogs/PetImages', transform=data_transforms)

## create validation

In [6]:
indices = random.sample(range(len(full_dataset)), 100)
train_subset = Subset(full_dataset, indices[:80])
val_subset = Subset(full_dataset, indices[80:])


## Data loader

In [7]:
train_loader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False, num_workers=2)


## verification

In [8]:
dataset_sizes = {"train": len(train_subset), "val": len(val_subset)}
class_names = full_dataset.classes

print(dataset_sizes)
print(class_names)

{'train': 80, 'val': 20}
['Cat', 'Dog']


# Pretrained Model: ResNet-18 on the small dataset

- Load **ResNet-18** pretrained on ImageNet
- Replace the final fully connected layer to match the number of classes in the small dataset (2: Cat, Dog)

In [9]:
import torch
import torch.nn as nn
from torchvision import models


In [10]:
%%capture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

print(model)

## Define loss and Optimizer

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

## Freeze Backbone

In [12]:
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

## Training the Model (Small Dataset, Freeze Backbone)

- Train only the classifier head (backbone frozen)
- Track training time
- Evaluate validation accuracy after each epoch


In [13]:
import time
import copy

def train_model(model, dataloaders, criterion, optimizer, num_epochs=5):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

## Train Small Dataset (Freeze Backbone)

In [14]:
model = train_model(model, {"train": train_loader, "val": val_loader}, criterion, optimizer, num_epochs=5)

Epoch 1/5
train Loss: 0.9103 Acc: 0.3625
val Loss: 0.5632 Acc: 0.8000
Epoch 2/5
train Loss: 0.6642 Acc: 0.6000
val Loss: 0.3985 Acc: 0.9000
Epoch 3/5
train Loss: 0.4129 Acc: 0.8875
val Loss: 0.4004 Acc: 0.8500
Epoch 4/5
train Loss: 0.2873 Acc: 0.9250
val Loss: 0.2094 Acc: 0.9500
Epoch 5/5
train Loss: 0.2331 Acc: 0.9375
val Loss: 0.1723 Acc: 0.9500
Training complete in 0m 53s
Best val Acc: 0.9500


## Training the Model (Small Dataset, Fine-tune Entire Model)

- All layers are trainable
- Compare performance and training time with frozen-backbone approach


In [15]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [16]:
model = train_model(model, {"train": train_loader, "val": val_loader}, criterion, optimizer, num_epochs=5)


Epoch 1/5
train Loss: 0.7904 Acc: 0.4875
val Loss: 0.5548 Acc: 0.7000
Epoch 2/5
train Loss: 0.4924 Acc: 0.7875
val Loss: 0.3335 Acc: 0.9000
Epoch 3/5
train Loss: 0.3139 Acc: 0.9000
val Loss: 0.2470 Acc: 0.9500
Epoch 4/5
train Loss: 0.1515 Acc: 0.9500
val Loss: 0.1523 Acc: 1.0000
Epoch 5/5
train Loss: 0.0868 Acc: 1.0000
val Loss: 0.1239 Acc: 1.0000
Training complete in 2m 5s
Best val Acc: 1.0000


# Large Dataset (10K+),

In [17]:
!mkdir -p ./data/intel
!kaggle datasets download -d puneet6060/intel-image-classification -p ./data/intel
!unzip -q ./data/intel/intel-image-classification.zip -d ./data/intel

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
replace ./data/intel/seg_pred/seg_pred/10004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [18]:
large_train_dataset = datasets.ImageFolder(root='./data/intel/seg_train/seg_train', transform=data_transforms)
large_val_dataset = datasets.ImageFolder(root='./data/intel/seg_test/seg_test', transform=data_transforms)

train_loader = DataLoader(large_train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(large_val_dataset, batch_size=32, shuffle=False, num_workers=2)

dataset_sizes = {"train": len(large_train_dataset), "val": len(large_val_dataset)}
class_names = large_train_dataset.classes

print(dataset_sizes)
print(class_names)

{'train': 14034, 'val': 3000}
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [19]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)


In [20]:
model = train_model(model, {"train": train_loader, "val": val_loader}, criterion, optimizer, num_epochs=5)


Epoch 1/5
train Loss: 0.5170 Acc: 0.8332
val Loss: 0.3044 Acc: 0.8963
Epoch 2/5
train Loss: 0.3235 Acc: 0.8883
val Loss: 0.2777 Acc: 0.9010
Epoch 3/5
train Loss: 0.2981 Acc: 0.8958
val Loss: 0.2665 Acc: 0.9013
Epoch 4/5
train Loss: 0.2889 Acc: 0.8972
val Loss: 0.2543 Acc: 0.9090
Epoch 5/5
train Loss: 0.2756 Acc: 0.9034
val Loss: 0.2593 Acc: 0.9047
Training complete in 129m 25s
Best val Acc: 0.9090


In [21]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)